# Fine tuning with Azure Open AI

Documentation:
https://learn.microsoft.com/en-us/azure/ai-services/openai/tutorials/fine-tune?tabs=command-line

Information:
https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models#fine-tuning-models-preview

In [103]:
import datetime
import openai
import os
import json
import numpy as np
import requests
import sys
import time
import tiktoken

from dotenv import load_dotenv
from IPython.display import clear_output

In [2]:
load_dotenv("azure.env")

openai.api_type: str = "azure"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = "2023-09-15-preview"

In [3]:
sys.version

'3.10.10 (main, Mar 21 2023, 18:45:11) [GCC 11.2.0]'

In [4]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 23-Oct-2023 07:49:58


In [5]:
print("Open AI version:", openai.__version__)

Open AI version: 0.28.1


## Testing

In [6]:
model = "gpt-35-turbo-16k"

In [7]:
response = openai.ChatCompletion.create(
    engine=model,
    messages=[{"role": "user", "content": "Hello"}],
    max_tokens=100,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)

print("Response:\n\n", type(response), response)

Response:

 <class 'openai.openai_object.OpenAIObject'> {
  "id": "chatcmpl-8CjzGbUtqE5LO9iWYTSkHI1LajZNB",
  "object": "chat.completion",
  "created": 1698047402,
  "model": "gpt-35-turbo-16k",
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
       

In [8]:
print("Answer type:\n\n", type(response.choices[0].message))

Answer type:

 <class 'openai.openai_object.OpenAIObject'>


In [9]:
print("Answer:\n\n", response.choices[0].message)

Answer:

 {
  "role": "assistant",
  "content": "Hello! How can I assist you today?"
}


## Training and validation Jsonl files

In [10]:
training_file_name = "training_set.jsonl"
validation_file_name = "validation_set.jsonl"

In [11]:
!ls $training_file_name -lh

-rwxrwxrwx 1 root root 2.7K Oct 17 15:35 training_set.jsonl


In [12]:
with open(training_file_name, "r") as f:
    print(f.read())

{"messages": [{"role": "system", "content": "Clippy is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who discovered Antarctica?"}, {"role": "assistant", "content": "Some chaps named Fabian Gottlieb von Bellingshausen and Mikhail Lazarev, as if they don't teach that in every school!"}]}
{"messages": [{"role": "system", "content": "Clippy is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What is the biggest ocean?"}, {"role": "assistant", "content": "The Pacific Ocean. It's not like it's a small pond or anything."}]}
{"messages": [{"role": "system", "content": "Clippy is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What is the largest planet?"}, {"role": "assistant", "content": "It's called Jupiter, you may have heard of it...or not."}]}
{"messages": [{"role": "system", "content": "Clippy is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who painted the Mona Lisa?"}, {"role": "a

In [13]:
!ls $validation_file_name -lh

-rwxrwxrwx 1 root root 2.6K Oct 17 15:35 validation_set.jsonl


In [14]:
with open(validation_file_name, "r") as f:
    print(f.read())

{"messages": [{"role": "system", "content": "Clippy is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of Australia?"}, {"role": "assistant", "content": "It's Canberra, not Sydney. Shocking, I know!"}]}
{"messages": [{"role": "system", "content": "Clippy is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who wrote 'The Great Gatsby'?"}, {"role": "assistant", "content": "Some guy named F. Scott Fitzgerald. Ever heard of him?"}]}
{"messages": [{"role": "system", "content": "Clippy is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the deepest part of the ocean?"}, {"role": "assistant", "content": "Mariana Trench, makes your swimming pool seem a bit shallow, doesn't it?"}]}
{"messages": [{"role": "system", "content": "Clippy is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who composed the 'Moonlight Sonata'?"}, {"role": "assistant", "content": "Just a little

## Fine tuning

In [15]:
# Load the training set
with open(training_file_name, "r", encoding="utf-8") as f:
    training_dataset = [json.loads(line) for line in f]

# Training dataset stats
print("Number of examples in training set:", len(training_dataset))
print("First example in training set:")
for message in training_dataset[0]["messages"]:
    print(message)

Number of examples in training set: 10
First example in training set:
{'role': 'system', 'content': 'Clippy is a factual chatbot that is also sarcastic.'}
{'role': 'user', 'content': 'Who discovered Antarctica?'}
{'role': 'assistant', 'content': "Some chaps named Fabian Gottlieb von Bellingshausen and Mikhail Lazarev, as if they don't teach that in every school!"}


In [16]:
# Load the validation set
with open(validation_file_name, "r", encoding="utf-8") as f:
    validation_dataset = [json.loads(line) for line in f]

# Validation dataset stats
print("\nNumber of examples in validation set:", len(validation_dataset))
print("First example in validation set:")
for message in validation_dataset[0]["messages"]:
    print(message)


Number of examples in validation set: 10
First example in validation set:
{'role': 'system', 'content': 'Clippy is a factual chatbot that is also sarcastic.'}
{'role': 'user', 'content': "What's the capital of Australia?"}
{'role': 'assistant', 'content': "It's Canberra, not Sydney. Shocking, I know!"}


## Tokens information

In [17]:
encoding = tiktoken.get_encoding("cl100k_base")

In [18]:
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    """
    Number of tokens from messages
    """
    num_tokens = 0

    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name

    num_tokens += 3

    return num_tokens

In [19]:
def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))

    return num_tokens

In [20]:
def print_distribution(values, name):
    """
    Stats
    """
    print(f"Distribution of {name}:")
    print(f"   min / max: {min(values)}, {max(values)}")
    print(f"   mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"   p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")
    print()

In [21]:
files = [training_file_name, validation_file_name]

for file in files:
    print()
    print("-" * 100)
    print(f"Processing file: {file}")
    print("-" * 100)

    with open(file, "r", encoding="utf-8") as f:
        dataset = [json.loads(line) for line in f]

    total_tokens = []
    assistant_tokens = []

    for ex in dataset:
        messages = ex.get("messages", {})
        total_tokens.append(num_tokens_from_messages(messages))
        assistant_tokens.append(num_assistant_tokens_from_messages(messages))

    print_distribution(total_tokens, "total tokens")
    print_distribution(assistant_tokens, "assistant tokens")


----------------------------------------------------------------------------------------------------
Processing file: training_set.jsonl
----------------------------------------------------------------------------------------------------
Distribution of total tokens:
   min / max: 47, 62
   mean / median: 52.1, 50.5
   p5 / p95: 47.9, 57.5

Distribution of assistant tokens:
   min / max: 13, 30
   mean / median: 17.6, 15.5
   p5 / p95: 13.0, 21.9


----------------------------------------------------------------------------------------------------
Processing file: validation_set.jsonl
----------------------------------------------------------------------------------------------------
Distribution of total tokens:
   min / max: 43, 65
   mean / median: 51.4, 49.0
   p5 / p95: 45.7, 56.9

Distribution of assistant tokens:
   min / max: 8, 29
   mean / median: 15.9, 13.5
   p5 / p95: 11.6, 20.9



In [22]:
total_tokens, assistant_tokens

([48, 49, 56, 49, 65, 46, 56, 54, 48, 43],
 [13, 13, 19, 13, 29, 12, 18, 20, 14, 8])

## Running the fine tuning model job

In [23]:
training_response = openai.File.create(
    file=open(training_file_name, "rb"),
    purpose="fine-tune",
    user_provided_filename=training_file_name,
)
training_file_id = training_response["id"]


validation_response = openai.File.create(
    file=open(validation_file_name, "rb"),
    purpose="fine-tune",
    user_provided_filename=validation_file_name,
)
validation_file_id = validation_response["id"]

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-aa18bebf22194443bcb81d48bc4434ca
Validation file ID: file-bbc5e0d7274c40d183333e1839a6d43a


# Fine tuning model

**babbage-002** and **davinci-002** are not trained to follow instructions. Querying these base models should only be done as a point of reference to a fine-tuned version to evaluate the progress of your training.

**gpt-35-turbo-0613** - fine-tuning of this model is limited to a subset of regions, and is not available in every region the base model is available.

In [24]:
ftmodel = "gpt-35-turbo-0613"

In [26]:
response = openai.FineTuningJob.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model=ftmodel,
)

job_id = response["id"]

print("Job ID:", response["id"])
print("Status:", response["status"])
print(response)

Job ID: ftjob-98c5a54e805a42678eb3286f52d76e1b
Status: pending
{
  "hyperparameters": {
    "n_epochs": 2
  },
  "status": "pending",
  "model": "gpt-35-turbo-0613",
  "training_file": "file-aa18bebf22194443bcb81d48bc4434ca",
  "validation_file": "file-bbc5e0d7274c40d183333e1839a6d43a",
  "id": "ftjob-98c5a54e805a42678eb3286f52d76e1b",
  "created_at": 1698047499,
  "updated_at": 1698047499,
  "object": "fine_tuning.job"
}


In [43]:
def check_training_status():
    """
    Check fine tuning process
    """
    response = openai.FineTuningJob.retrieve(job_id)

    if response["status"] == "pending":
        print(f"Fine tuning of {ftmodel} is PENDING - Please wait")

    elif response["status"] == "running":
        print("\033[1;31;34m")
        print(f"Fine tuning of {ftmodel} is RUNNING - Please wait")

    elif response["status"] == "succeeded":
        print("\033[1;31;32m")
        print(f"Fine tuning of {ftmodel} is DONE!")

    else:
        print(response)

    print("\nJob ID:", job_id)
    print("Created:", datetime.datetime.utcfromtimestamp(response["created_at"]))
    print("Updated:", datetime.datetime.utcfromtimestamp(response["updated_at"]))

    print("\033[0m")
    print(response)

In [28]:
check_training_status()

Fine tuning of gpt-35-turbo-0613 is PENDING - Please wait

Job ID: ftjob-98c5a54e805a42678eb3286f52d76e1b
Created: 2023-10-23 07:51:39
Updated: 2023-10-23 07:51:39

{
  "hyperparameters": {
    "n_epochs": 2
  },
  "status": "pending",
  "model": "gpt-35-turbo-0613",
  "training_file": "file-aa18bebf22194443bcb81d48bc4434ca",
  "validation_file": "file-bbc5e0d7274c40d183333e1839a6d43a",
  "id": "ftjob-98c5a54e805a42678eb3286f52d76e1b",
  "created_at": 1698047499,
  "updated_at": 1698047499,
  "object": "fine_tuning.job"
}


In [29]:
check_training_status()


Fine tuning of gpt-35-turbo-0613 is RUNNING - Please wait

Job ID: ftjob-98c5a54e805a42678eb3286f52d76e1b
Created: 2023-10-23 07:51:39
Updated: 2023-10-23 07:52:26

{
  "hyperparameters": {
    "n_epochs": 2
  },
  "status": "running",
  "model": "gpt-35-turbo-0613",
  "training_file": "file-aa18bebf22194443bcb81d48bc4434ca",
  "validation_file": "file-bbc5e0d7274c40d183333e1839a6d43a",
  "id": "ftjob-98c5a54e805a42678eb3286f52d76e1b",
  "created_at": 1698047499,
  "updated_at": 1698047546,
  "object": "fine_tuning.job"
}


In [44]:
check_training_status()


Fine tuning of gpt-35-turbo-0613 is DONE!

Job ID: ftjob-98c5a54e805a42678eb3286f52d76e1b
Created: 2023-10-23 07:51:39
Updated: 2023-10-23 08:39:43

{
  "hyperparameters": {
    "n_epochs": 2
  },
  "status": "succeeded",
  "model": "gpt-35-turbo-0613",
  "fine_tuned_model": "gpt-35-turbo-0613.ft-98c5a54e805a42678eb3286f52d76e1b",
  "training_file": "file-aa18bebf22194443bcb81d48bc4434ca",
  "validation_file": "file-bbc5e0d7274c40d183333e1839a6d43a",
  "result_files": [
    "file-baebd298379b4f858922cb205345f5a4"
  ],
  "finished_at": 1698050383,
  "trained_tokens": 1048,
  "id": "ftjob-98c5a54e805a42678eb3286f52d76e1b",
  "created_at": 1698047499,
  "updated_at": 1698050383,
  "object": "fine_tuning.job"
}


> https://oai.azure.com/portal

In [45]:
openai.FineTuningJob.list()

<OpenAIObject list at 0x7ff2e3751da0> JSON: {
  "has_more": false,
  "data": [
    {
      "hyperparameters": {
        "n_epochs": 2
      },
      "status": "succeeded",
      "model": "gpt-35-turbo-0613",
      "fine_tuned_model": "gpt-35-turbo-0613.ft-98c5a54e805a42678eb3286f52d76e1b",
      "training_file": "file-aa18bebf22194443bcb81d48bc4434ca",
      "validation_file": "file-bbc5e0d7274c40d183333e1839a6d43a",
      "result_files": [
        "file-baebd298379b4f858922cb205345f5a4"
      ],
      "finished_at": 1698050383,
      "trained_tokens": 1048,
      "id": "ftjob-98c5a54e805a42678eb3286f52d76e1b",
      "created_at": 1698047499,
      "updated_at": 1698050383,
      "object": "fine_tuning.job"
    }
  ],
  "object": "list"
}

<img src="capture1.png">

## Deploying the model

- token:	There are multiple ways to generate an authorization token. The easiest method for initial testing is to launch the Cloud Shell from the Azure portal. Then run az account get-access-token. You can use this token as your temporary authorization token for API testing. We recommend storing this in a new environment variable
- subscription:	The subscription ID for the associated Azure OpenAI resource
- resource_group:	The resource group name for your Azure OpenAI resource
- resource_name:	The Azure OpenAI resource name
- model_deployment_name:	The custom name for your new fine-tuned model deployment. This is the name that will be referenced in your code when making chat completion calls.

In [46]:
# Retrieve fine_tuned_model name
response = openai.FineTuningJob.retrieve(job_id)

print(response)
fine_tuned_model = response["fine_tuned_model"]

{
  "hyperparameters": {
    "n_epochs": 2
  },
  "status": "succeeded",
  "model": "gpt-35-turbo-0613",
  "fine_tuned_model": "gpt-35-turbo-0613.ft-98c5a54e805a42678eb3286f52d76e1b",
  "training_file": "file-aa18bebf22194443bcb81d48bc4434ca",
  "validation_file": "file-bbc5e0d7274c40d183333e1839a6d43a",
  "result_files": [
    "file-baebd298379b4f858922cb205345f5a4"
  ],
  "finished_at": 1698050383,
  "trained_tokens": 1048,
  "id": "ftjob-98c5a54e805a42678eb3286f52d76e1b",
  "created_at": 1698047499,
  "updated_at": 1698050383,
  "object": "fine_tuning.job"
}


In [65]:
token = os.getenv("TOKEN")
subscription = os.getenv("SUBSCRIPTION")
resource_group = os.getenv("RESOURCE_GROUP")
resource_name = os.getenv("RESOURCE_NAME")

model_deployment_name = "mymodel"

In [97]:
deploy_params = {"api-version": "2023-05-01"}
deploy_headers = {
    "Authorization": "Bearer {}".format(token),
    "Content-Type": "application/json",
}

deploy_data = {
    "sku": {"name": "standard", "capacity": 1},
    "properties": {
        "model": {
            "format": "OpenAI",
            "name": "gpt-35-turbo-0613.ft-98c5a54e805a42678eb3286f52d76e1b",  # retrieve this value from the previous call
            "version": "1",
        }
    },
}

deploy_data = json.dumps(deploy_data)
request_url = f"https://management.azure.com/subscriptions/{subscription}/resourceGroups/{resource_group}/providers/Microsoft.CognitiveServices/accounts/{resource_name}/deployments/{model_deployment_name}"


In [98]:
deploy_data

'{"sku": {"name": "standard", "capacity": 1}, "properties": {"model": {"format": "OpenAI", "name": "gpt-35-turbo-0613.ft-98c5a54e805a42678eb3286f52d76e1b", "version": "1"}}}'

In [99]:
print("Creating a new deployment...")
resp = requests.put(
    request_url, params=deploy_params, headers=deploy_headers, data=deploy_data
)

print(resp)

Creating a new deployment...
<Response [201]>


In [111]:
print(resp.reason)

Created


In [130]:
mymodel = resp.json()["name"]
mymodel

'mymodel'

In [126]:
resp.json()["systemData"]["createdAt"]

'2023-10-23T09:10:34.1557354Z'

In [127]:
resp.json()["properties"]["model"]

{'format': 'OpenAI', 'name': 'gpt-35-turbo-0613.ft-98c5a54e805a42678eb3286f52d76e1b', 'version': '1'}

In [134]:
status = resp.json()["properties"]["provisioningState"]

print(f"Model deployment status of model '{mymodel}': {status}")

Model deployment status of model 'mymodel': Creating


In [153]:
resp = requests.put(
    request_url, params=deploy_params, headers=deploy_headers, data=deploy_data
)

status = resp.json()["properties"]["provisioningState"]
print(f"Model deployment status of model '{mymodel}': {status}")

Model deployment status of model 'mymodel': Succeeded


## Testing the model

In [146]:
openai.api_version = "2023-05-15"

In [148]:
response = openai.ChatCompletion.create(
    engine=mymodel, 
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {
            "role": "assistant",
            "content": "Yes, customer managed keys are supported by Azure OpenAI.",
        },
        {"role": "user", "content": "Do other Azure AI services support this too?"},
    ],
)

In [149]:
print(response)

{
  "id": "chatcmpl-8ClnlcIlMElGYVmqD2phNQG9lxWdU",
  "object": "chat.completion",
  "created": 1698054377,
  "model": "gpt-35-turbo-0613.ft-98c5a54e805a42678eb3286f52d76e1b",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "Yes, other Azure AI services also support customer managed keys. This allows customers to have more control over the encryption and security of their data."
      }
    }
  ],
  "usage": {
    "prompt_tokens": 55,
    "completion_tokens": 28,
    "total_tokens": 83
  }
}


In [150]:
print("Answer type:\n\n", type(response.choices[0].message))

Answer type:

 <class 'openai.openai_object.OpenAIObject'>


In [151]:
print(response["choices"][0]["message"]["content"])

Yes, other Azure AI services also support customer managed keys. This allows customers to have more control over the encryption and security of their data.
